# Рекомендация тарифов

## Изучение файла

In [1]:
# импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# чтение файла с данными и сохранение в df с использованием конструкции try-except
try:
    df = pd.read_csv('C:/Users/datasets/users_behavior.csv')  # локальный путь
except:
    df = pd.read_csv('/datasets/users_behavior.csv')  # серверный путь

In [3]:
# получение 5 строк таблицы df
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
1230,100.0,639.43,42.0,11230.93,0
2996,67.0,499.29,10.0,23124.48,0
1737,35.0,222.44,6.0,4659.43,0
3201,56.0,419.42,59.0,5177.62,1
1280,3.0,12.70,0.0,492.03,1


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
# получение статистической информации
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Таким образом, таблица состоит из 5 числовых столбцов. В каждой строке таблицы - данные о поведении клиентов. Количество значений в столбцах одинаковое, а значит, в данных нет пропущенных значений. Целевой признак (`is_ultra`) - это категориальная переменная (тариф ультра или нет), то в исследовании решается задача классификации.

## Разбиение данных на выборки

In [6]:
# создание переменных с признакими и целевым признаком
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [7]:
# разделение данных на обучающую, валидационную и тестовую выборки
features_train, features_test_val, target_train, target_test_val = (
    train_test_split(features, target, test_size=0.4, random_state=12345)) # разделям на обучающую и на проверочную выборки
features_valid, features_test, target_valid, target_test = (
    train_test_split(features_test_val, target_test_val, test_size=0.5, random_state=12345)) # проверочную делим на валидационную и тестовую

Получилось три выборки в соотношении 3:1:1. То есть на обучающую выборку отведено 60%, на валидационную и тестовую по 20%.

In [8]:
# проверка размеров полученных наборов
sets = [features_train, target_train, features_valid, target_valid, features_test, target_test]
for elem in sets:
    print(elem.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Исследование моделей

Так как мы имеем дело с задачей классификации, будем использовать три модели: решающее дерево, случайный лес и логистическая регрессия. 

In [9]:
# модель DecisionTreeClassifier
best_model = None
best_result = 0
best_depth = 0
for depth in range(1,6): # создаем цикл для перебора гиперпараметров
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
    predictions = model.predict(features_valid) # получаем предсказания модели на валидационной выборке
    result = accuracy_score(target_valid, predictions) # рассчитываем качество модели
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print('Accuracy наилучшей модели на валидационной выборке:', best_result, ',', 'max_depth =', best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 , max_depth = 3


In [10]:
# модель RandomForestClassifier
best_model = None
best_result = 0
best_depth = 0
best_est = 0
for est in range(1,11): 
    for depth in range(1,6):
        model = RandomForestClassifier(random_state=12345, n_estimators=est,  max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions)  
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print('Accuracy наилучшей модели на валидационной выборке:', best_result, ',', 'n_estimators =', best_est, ',', 'max_depth =', best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7931570762052877 , n_estimators = 8 , max_depth = 5


In [11]:
# модель LogisticRegression
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print('Accuracy модели логистической регрессии на валидационной выборке:', result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Модель RandomForestClassifier показала лучшее значение accuracy - 0.7931 при этом глубина дерева равна 5, а количество деревьев - 8. Чуть хуже оказалось значение accuracy у DecisionTreeClassifier (0.7854) при глубине дерева равное 3. И наименьшее значение accuracy равное 0.7589 у модели LogisticRegression.

## Проверка модели на тестовой выборке

In [12]:
# проверка качества RandomForestClassifier на тестовой выборке
model = RandomForestClassifier(random_state=12345, n_estimators=8,  max_depth=5)
model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
predictions_test = model.predict(features_test) # получаем предсказания модели на тестовой выборке
accuracy_score(target_test, predictions_test)

0.7962674961119751

In [13]:
# проверка качества DecisionTreeClassifier на тестовой выборке
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

0.7791601866251944

In [14]:
# проверка качества LogisticRegression на тестовой выборке
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
accuracy_score(target_test, predictions_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7402799377916018

Проверив качество моделей на тестовой выборке, мы видим, что для модели RandomForestClassifier результат оказался даже немного выше (0.7963), для остальных же моделей результат получился немного хуже, чем при проверке на валидацонной выборке.

**Вывод**

Была построена модель для задачи классификации, которая выберет подходящий тариф. Для обучения и оценки качества мы разделили данные на обучающую, валидационную и тестовую выборки. Построив три модели (решающее дерево, случайный лес и логистическая регрессия), была выбрана модель случайного леса (RandomForestClassifier), которая показала лучшее значение accuracy - 0.7931. При проверке качества данной модели на тестовой выборке результат оказался чуть выше (0.7963). 